In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

from energymodel import (
    EnergyModel, EMSolver, random_uniform, NanMonitor, Lyapunov
)

tf.compat.v1.reset_default_graph()

## Dataset

In [ ]:
def resample(batch_size):
    return random_uniform([batch_size, 3])  # 3D dynamics.


def to_tensor(x):
    return tf.convert_to_tensor(x, dtype='float32')


class LorenzDynamics:
    """The Lorenz dynamics, famous for its chaotic property.
    
    The usual setting of paramters is `sigma = 10`, `beta = 8/3`,
    and `rho` is adjustable. With this setting, the three phases
    of the dynamics are:

        1. `0 < rho < 1`: single stable attractor at the origin;
        2. `1 < rho < 24.74`: double stable attractor;
        3. `rho > 24.74`: strange attractors, i.e. chaotic behavior.

    Reference:
        [Wikipedia: Lorenz system](https://en.wikipedia.org/wiki/Lorenz_system).  # noqa:E501
    """
    
    def __init__(self,
                 sigma: float,
                 beta: float,
                 rho: float,
                 scale: float = None):
        """
        Args:
            sigma: The Lorenz parameter.
            beta: The Lorenz parameter.
            rho: The Lorenz parameter.
            scale: The input variable, generally standarized to (-1, 1),
                can be scaled by multiplying `scale`, so as to fit the
                real scale in the Lorenz dynamics. The output will be
                scaled back by dividing `scale`. Defaults to `1`.
        """
        self.sigma = to_tensor(sigma)
        self.beta = to_tensor(beta)
        self.rho = to_tensor(rho)

        if scale is None:
            scale = 1
        self.scale = to_tensor(scale)

    def _std_dynamics(self, v):
        """The standard Lorenz dynamics."""
        x, y, z = tf.unstack(v, axis=1)

        dx = self.sigma * (y - x)
        dy = x * (self.rho - z) - y
        dz = x * y - self.beta * z

        return tf.stack([dx, dy, dz], axis=1)

    def __call__(self, v):
        return self._std_dynamics(v * self.scale) / self.scale

In [ ]:
batch_size = 128
solver = EMSolver(dt=1e-2, eps=1e-2)
vector_field = LorenzDynamics(10, 8/3, 28, scale=5)
lyapunov = Lyapunov(
    vector_field,
    resample,
    solver,
    t=1e-0,
    T=1e-2,
)

In [ ]:
plot_lyapunov = lyapunov(batch_size * 10).numpy()

In [ ]:
plt.scatter(plot_lyapunov[:, 0], plot_lyapunov[:, 1], alpha=0.2)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(plot_lyapunov[:, 0], plot_lyapunov[:, 1], plot_lyapunov[:, 2], alpha=0.2)
plt.show()

## Model

In [ ]:
network = models.Sequential([
    layers.Dense(256),
    layers.Activation('swish'),

    layers.Dense(256),
    layers.Activation('swish'),

    layers.Dense(64),
    layers.Activation('swish'),

    layers.Dense(1, use_bias=False),
])
network(resample(batch_size))  # build.

model = EnergyModel(
    network,
    resample,
    solver,
    t=1e-0,
)
tf.print('T =', model.T)

optimizer = tf.optimizers.Adam(learning_rate=1e-3, clipvalue=1e-1)
callbacks = [
    NanMonitor(50),
]

train_step = model.get_optimize_fn(optimizer, callbacks)
train_step = tf.function(train_step)

In [ ]:
for step in tqdm(range(5000)):
    batch = lyapunov(batch_size)
    train_step(batch)

## Evaluation

- [ ] <font color="red">How to properly visualize the 3D dynamics?</font>

In [ ]:
fig = plt.figure(figsize=(12, 8))

# Contour Plot
num_grids = 30
# Determine the range by the previous 3D plot.
plot_X, plot_Y = np.meshgrid(
    np.linspace(-3, 3, num_grids),
    np.linspace(-3, 3, num_grids),
)
# Since it's a 3D dynamics, the Z-axis shall be determined.
# We use constant Z.
plot_batch = np.stack(
    [
        plot_X.reshape([-1]).astype('float32'),
        plot_Y.reshape([-1]).astype('float32'),
        2 * np.ones_like(plot_Y).reshape([-1]).astype('float32'),
    ],
    axis=1)
network_vals = network(plot_batch).numpy().reshape(num_grids, num_grids)
plt.contourf(plot_X, plot_Y, network_vals, 50, alpha=0.5)
plt.colorbar()

# Vector Plot
vec = vector_field(plot_batch).numpy().reshape(num_grids, num_grids, 3)
# vec = np.sign(vec)
vec_X, vec_Y = vec[:, :, 0], vec[:, :, 1]
plt.quiver(plot_X, plot_Y, vec_X, vec_Y)

plt.xlabel('x')
plt.ylabel('y')

plt.title('Network and Vector Field')
plt.show()

In [ ]:
def dot(x, y):
    return tf.reduce_sum(x * y, axis=1)


def criterion(test_batch):
    return dot(model.vector_field(test_batch), vector_field(test_batch))

In [ ]:
plot_criterion = criterion(tf.convert_to_tensor(plot_batch))
plot_criterion = plot_criterion.numpy().reshape([num_grids, num_grids])

plt.hist(plot_criterion.reshape([-1]), bins=100)
plt.title('Criterion on Grids')
plt.show()

print('Minimum =', plot_criterion.min())

In [ ]:
plt.contourf(plot_X, plot_Y, plot_criterion, 50, alpha=0.5)
plt.colorbar()
plt.title('Criterion on Grids')
plt.show()